## Récupérer les coordonnées des centres de distribution

In [38]:
import pandas as pd

df = pd.read_excel("data/centres.xlsx")

In [39]:
names = df["Nom"].tolist()
adresses = df["Adresse"].tolist()

In [40]:
import requests

api_key = open("api_key", "r").read()

In [41]:
def get_coords(addresses):
    latitudes = []
    longitudes = []

    for address in addresses:
        response = requests.get(
            f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
        )
        data = response.json()

        if data["status"] == "OK":
            location = data["results"][0]["geometry"]["location"]
            latitudes.append(location["lat"])
            longitudes.append(location["lng"])
        else:
            print(f"Geocoding failed for address {address}. Status:", data["status"])
    
    return latitudes, longitudes

In [42]:

latitudes, longitudes = get_coords(adresses)

In [43]:
for n, lat, long in zip(names, latitudes, longitudes) :
    print(    f"{n} : {lat}, {long}")

Toulouse/Seminaire : 43.6391233, 1.4356993
Auterive : 43.3518938, 1.479144
Bagneres de Luchon : 42.7911527, 0.5881116
Bessieres : 43.8006816, 1.6040233
Blagnac : 43.6467573, 1.3867645
Carbonne : 43.2971934, 1.2206374
Cazeres : 43.2103963, 1.1059544
Cugnaux : 43.5347748, 1.3502449
Escalquens : 43.517648, 1.5575128
Fenouillet : 43.6848225, 1.4029819
Fonsorbes : 43.5389381, 1.2351879
Fronton : 43.8381611, 1.3908275
L Isle en Dodon : 43.376577, 0.8372039
Leguevin : 43.599054, 1.236362
Levignac : 43.66709549999999, 1.1934514
Montrejeau : 43.0846413, 0.5626481
Muret : 43.47820900000001, 1.341775
Pibrac : 43.6218265, 1.2918542
Plaisance du Touch : 43.567516, 1.3065979
Portet sur Garonne : 43.5418696, 1.4085209
Revel : 43.45654649999999, 2.0037021
Rieumes : 43.4133038, 1.1190482
Saint-Gaudens : 43.106199, 0.7262609
Salies du Salat/Mane : 43.0834612, 0.9469247
Toulouse/Grande-Bretagne(Casselardit) : 43.60352750000001, 1.4076353
Toulouse/Malepere : 43.56513890000001, 1.5106988
Toulouse/Negogouss

In [35]:
# Add the latitudes as new columns into the original excel file
df["Latitude"] = latitudes
df["Longitude"] = longitudes

In [36]:
df.to_excel("data/centres.xlsx", index=False)

## Créer la matrice de distances

In [38]:
import googlemaps

gmaps = googlemaps.Client(key=api_key)

In [39]:
from tqdm import tqdm

matrix = []

for i, a in tqdm(enumerate(addresses), total=len(addresses)):
    matrix.append([])
    for j, b in enumerate(addresses):
        res = gmaps.distance_matrix(a, b)
        matrix[i].append(res["rows"][0]["elements"][0]["distance"]["value"])

100%|██████████| 29/29 [00:45<00:00,  1.58s/it]


In [40]:
mat = pd.DataFrame(matrix, columns=names, index=names)
mat.to_excel("data/matrix.xlsx")

## Récupérer les coordonnées des points de ramasse

In [9]:
pdr = pd.read_excel("data/centres.xlsx", sheet_name="Points de Ramasse")
print(pdr)

         Point de Ramasse                                            Adresse  \
0                   METRO     ZI la Glacière, 8 impasse Camo, 31018 Toulouse   
1          Auchan Gramont                        Auchan Hypermarché Toulouse   
2        Leclerc St Orens  5 All. des Champs Pinsons, 31650 Saint-Orens-d...   
3        Super U Flourens  Impasse du Moussard, 826 Rte de Castres D, 311...   
4         Leclerc Blagnac                   2 All. Emile Zola, 31715 Blagnac   
5        Leclerc Rouffiac         RD, 888 Rte d'Albi, 31180 Rouffiac-Tolosan   
6  Carrefour Supply Chain      1 Rue du Dr Charcot, 31830 Plaisance-du-Touch   

   Latitude  Longitude  Fréquence de Ramasse  
0       NaN        NaN                   NaN  
1       NaN        NaN                   NaN  
2       NaN        NaN                   NaN  
3       NaN        NaN                   NaN  
4       NaN        NaN                   NaN  
5       NaN        NaN                   NaN  
6       NaN        NaN        

In [10]:
adresses = pdr["Adresse"]
latitudes, longitudes = get_coords(adresses)

In [11]:
for lat, long in zip(latitudes, longitudes) :
    print(    f"{lat}, {long}")

43.6431035, 1.4160821
43.6319261, 1.4843222
43.5661466, 1.5171659
43.5864191, 1.5776498
43.644588, 1.3726111
43.6666046, 1.514161
43.5913663, 1.2960563


In [12]:
pdr["Latitude"] = latitudes
pdr["Longitude"] = longitudes
pdr.to_excel("data/centres.xlsx", sheet_name="Points de Ramasse", index=False)